### Plotting Utility Functions

In [ ]:
# Library requirements 
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
def plot_training_loss(hist):
    """
    Plots the training and validation loss over epochs from a training history object.

    Parameters:
    -----------
    hist : A History object returned by the `fit` method of a Keras model.

    Returns:
    --------
    None
       
    """
    
    plt.plot(hist.history['loss'], label='train_loss')
    plt.plot(hist.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def plot_pred_vs_test(y_test, y_pred, title, x_label, y_label):
    """
    Plots the predicted values against the true values, with a reference line y = x.

    Parameters:
    -----------
    y_test : array: true values (ground truth) to be compared against.
    y_pred : array: predicted values from the model.
    title : str: title of the plot.
    x_label : str: x-axis label.
    y_label : str: y-axis label.

    Returns:
    --------
    None
    """

    plt.figure(figsize=(10, 6))
    # True values (x-axis) vs Predicted values (y-axis)
    plt.scatter(y_test, y_pred, c='cyan', alpha=0.2, s=2, label='Predicted vs True')
    # Plot the line y = x for reference
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=1, label='y = x')

    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.title(title, fontsize=14)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.legend()
    plt.show()

In [ ]:

def plot_pe_rel(df, trade_type, plot_type, title, x_label, y_label, ax):
    """
    Plots the mean absolute or relative error for different mtm bins based on trade type.

    Parameters:
    -----------
    df : DataFrame: input DataFrame containing the data to plot. It must have columns 'y_test', 'diff_abs', and 'diff_rel'.
    trade_type : str: type of trade which determines the bin edges. ()'idx', 'idx_cs01', 'opt_delta', 'idx_theta', 'opt_theta', 'opt_deltastrike', 'opt_vega', 'opt')
    plot_type : str: type of plot to create. ()'abs' for mean absolute error, 'rel' for mean relative error)
    title : str: title of the plot.
    x_label : str: label for the x-axis.
    y_label : str: label for the y-axis.
    ax : matplotlib.axes.Axes:  matplotlib Axes object where the plot will be drawn.

    Returns:
    --------
    DataFrame: containing the mtm bins and their corresponding mean errors.
    """

    # Define mtm bins 
    if trade_type == 'idx':
        bin_edges = [-0.4, -0.35, -0.3, -0.25, -0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2]
    elif trade_type == 'idx_cs01':
        bin_edges = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
    elif trade_type == 'opt_delta':
        bin_edges = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5]
    elif trade_type == 'idx_theta':
        bin_edges = [ -0.006, -0.003666667, -0.00133,  0.001,  0.0033,  0.00566,  0.0080, 0.0103,  0.0126,  0.015]
    elif trade_type == 'opt_theta':
        bin_edges = [0.00025000, 0.01533333, 0.03091667, 0.04650000, 0.06208333, 0.07766667, 0.09325000, 0.10883333, 0.12441667, 0.14000000 ]
    elif trade_type == 'opt_deltastrike':
        bin_edges = [-4.81, -4.27261111, -3.73522222, -3.19783333, -2.66044444, -2.12305556, -1.58566667, -1.04827778, -0.51088889, -0.0055]
    elif trade_type == 'opt_vega':
        bin_edges = [0., 0.0009, 0.0018, 0.0027, 0.0036, 0.0045, 0.0054, 0.0063,  0.0072, 0.0081 ]
    elif trade_type == 'opt':
        bin_edges = [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2]
    df = df.copy()  
    df.loc[:, 'mtm_bin'] = pd.cut(df['y_test'], bins=bin_edges)

    # Group by mtm bins and calculate the mean absolute error for each bin
    if plot_type == 'abs':
        bin_errors = df.groupby('mtm_bin')['diff_abs'].mean().reset_index()
        ax.bar(bin_errors['mtm_bin'].astype(str), bin_errors['diff_abs'], color='lightblue')
    elif plot_type == 'rel':
        bin_errors = df.groupby('mtm_bin')['diff_rel'].mean().reset_index()
        ax.bar(bin_errors['mtm_bin'].astype(str), bin_errors['diff_rel'], color='lightblue')

    # Plot the histogram
    ax.set_xlabel(x_label, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)
    ax.set_title(title, fontsize=13)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=10)
    ax.grid(True)
    ax.tick_params(axis='y', labelsize=10)
    return bin_errors

def plot_pred_error_rel_twice(df1, df2, type, title1, title2, x=None):
    """
    Plots the mean relative and absolute errors for two side-by-side subplots.

    Parameters:
    -----------
    df1 : DataFrame: first input data DataFrame 
    df2 : DataFrame: second input data DataFrame
    type : str: type of trade which determines the bin edges for mtm  ()'idx', 'idx_cs01', 'opt_delta', 'idx_theta', 'opt_theta', 'opt_deltastrike', 'opt_vega', 'opt')
    title1 : str: title of the first subplot.
    title2 : str: title of the second subplot.
    x : str, optional: label for the x-axis. defaults to 'MtM Bin'.

    Returns:
    --------
    tuple
        A tuple containing two DataFrames:
        - bin_errors1 : DataFrame with mean relative errors from the first DataFrame.
        - bin_errors2 : DataFrame with mean raw errors from the second DataFrame.
    """

    # Set up the subplots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

    if x is None:
        x_lab = 'Price Bin'
    else:
        x_lab = x

    # Call the function twice for relative and abs diffs 
    bin_errors1 = plot_pe_rel(df1, type, 'rel', title=title1, x_label=x_lab, y_label='Mean Error (%)', ax=axes[0])
    bin_errors2 = plot_pe_rel(df2, type, 'abs', title=title2, x_label=x_lab, y_label='Mean Raw Error', ax=axes[1])
    plt.tight_layout()
    plt.show()

    return(bin_errors1, bin_errors2)


In [ ]:
def plot_pe(ax, df_, plot_type, trade_type, title, x_label, y_label, size=0.05):
     
    """
    Plots prediction errors against true values for a given trade type and plot type.

    Parameters:
    -----------
    ax : matplotlib.axes.Axes: Axes object where the plot will be drawn.
    df_ : DataFrame: containing the data to plot. It must have columns 'y_test', 'diff_abs', and 'diff_rel'.
    plot_type : str: type of plot to create. ('abs',  'rel') 
    trade_type : str: type of trade which determines the x-axis limits. ('idx', 'opt')
    title : str: title of the plot.
    x_label : str: label for the x-axis.
    y_label : str: label for the y-axis.
    size : float, optional: size of the scatter plot points. Default is 0.05.

    Returns:
    --------
    None
        
    """

    if plot_type == 'abs':
        y_data = df_['diff_abs']
    elif plot_type == 'rel':
        y_data = df_['diff_rel']

    # Plot the data
    ax.scatter(df_['y_test'], y_data, s=size, c='lightblue', alpha=0.2)
    ax.axhline(0, color='r', linestyle='--')
    ax.set_xlabel(x_label,fontsize=12)
    ax.set_ylabel(y_label,fontsize=12)
    ax.set_title(title, fontsize=13)
    if trade_type == 'idx':
        ax.set_xlim(-0.2, 0.2)
    elif trade_type == 'opt': 
        ax.set_xlim(0.0, 0.2)   
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)

def plot_pred_error_twice(df1, df2, trade_type, title1, title2, size=0.05, x=None):
    """
    Plots relative and absolute prediction errors for two dataframes using subplots.

    Parameters:
    -----------
    df1 : DataFrame: first input containing the data to plot.
    df2 : DataFrame: second input containing the data to plot.
    trade_type : str: type of trade which determines the x-axis limits. ('idx', 'opt')
    title1 : str: title of the first subplot.
    title2 : str: title of the second subplot.
    size : float, optional: size of the scatter plot points. Default is 0.05.
    x : str, optional: label for the x-axis. If not provided, defaults to 'Price'.

    Returns:
    --------
    None
        This function does not return any value. It displays two scatter plots of the prediction errors.
    """
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    if x is None:
        x_lab = 'Price'
    else:
        x_lab = x

    plot_pe(axes[0], df1, 'rel', trade_type, title1, x_lab, 'Relative Error (%)', size)
    plot_pe(axes[1], df2, 'abs', trade_type, title2, x_lab, 'Raw Error', size)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_index_dml_fit(y_true, y_pred, dydx_true, dydx_pred, name1, name2, name3):
    """
    Plots the predicted values against the true values and the predicted gradients against the true gradients
    for two dimensions, using subplots.

    Parameters:
    -----------
    y_true : array: true values (ground truth)
    y_pred : array: predicted NN values 
    dydx_true : DataFrame: true gradients (ground truth)
    dydx_pred : DataFrame: predicted gradients from the model.
    name1 : str: title for the first subplot (predicted vs true values for MTM).
    name2 : str: title for the second subplot (predicted vs true gradients for CS01).
    name3 : str: title for the third subplot (predicted vs true gradients for THETA).

    Returns:
    --------
    None
    
    """
    plt.figure(figsize=(21, 6))

    # Plot 1: y_pred vs y_true
    plt.subplot(1, 3, 1)
    plt.scatter(y_true, y_pred, s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Values', fontsize=14)
    plt.ylabel('Predicted Values', fontsize=14)
    plt.title(f'{name1}: pred vs true', fontsize=16)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Plot 2: dydx_pred vs dydx_true (dimension 0)
    plt.subplot(1, 3, 2)
    dimension = 0
    plt.scatter(dydx_true.iloc[:, dimension], dydx_pred[:, dimension], s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Gradients', fontsize=14)
    plt.ylabel('Predicted Gradients', fontsize=14)
    plt.title(f'{name2}: pred vs true ', fontsize=16)
    plt.plot([min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 
             [min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Plot 3: dydx_pred vs dydx_true (dimension 1)
    plt.subplot(1, 3, 3)
    dimension = 1
    plt.scatter(dydx_true.iloc[:, dimension], dydx_pred[:, dimension], s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Gradients', fontsize=14)
    plt.ylabel('Predicted Gradients', fontsize=14)
    plt.title(f'{name3}: pred vs true', fontsize=16)
    plt.plot([min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 
             [min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np 

def plot_error_bars(result, type, within_val_gt, within_val_nn):
    """
    Plots the absolute errors within and outside error bars for ground truth and predicted MtM values.

    Parameters:
    -----------
    result : DataFrame: input DataFrame containing the results data. It must have columns ()'mtm_t', 'mtm_t_pred', 'mtm_t_within_nn_error', and 'mtm_t_pred_within_nn_error')
    
    type : str: trade type ('Index', 'Option')

    Returns:
    --------
    None
 
    """
    # Create two plots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Within Error Bars ' + within_val_gt, markerfacecolor='lightgreen', markersize=10, alpha=0.5),
        Line2D([0], [0], marker='o', color='w', label='Outside Error Bars', markerfacecolor='purple', markersize=10, alpha=0.5)
    ]

    # First plot: using mtm_t_within_nn_error
    within_error = result[result['mtm_t_within_nn_error']]
    outside_error = result[~result['mtm_t_within_nn_error']]

    #ax1.scatter(within_error['mtm_t'], np.abs(within_error['mtm_t'] - within_error['mtm_t_pred']), s=0.5, c='lightgreen', alpha=0.5)
    #ax1.scatter(outside_error['mtm_t'], np.abs(outside_error['mtm_t'] - outside_error['mtm_t_pred']), s=0.5, c='purple', alpha=0.5)
    ax1.scatter(within_error['mtm_t'], within_error['mtm_t'] - within_error['mtm_t_pred'], s=0.5, c='lightgreen', alpha=0.5)
    ax1.scatter(outside_error['mtm_t'], outside_error['mtm_t'] - outside_error['mtm_t_pred'], s=0.5, c='purple', alpha=0.5)
    ax1.set_xlabel('Price', fontsize=11)
    ax1.set_ylabel('Raw Errors', fontsize=11)
    ax1.legend(handles=legend_elements)
    ax1.set_title(f'{type} Ground Truth Price within Error Bars',  fontsize=13)

    # Second plot: using mtm_t_pred_within_nn_error
    within_error = result[result['mtm_t_pred_within_nn_error']]
    outside_error = result[~result['mtm_t_pred_within_nn_error']]

    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Within Error Bars ' + within_val_nn, markerfacecolor='lightgreen', markersize=10, alpha=0.5),
        Line2D([0], [0], marker='o', color='w', label='Outside Error Bars', markerfacecolor='purple', markersize=10, alpha=0.5)
    ]

    #ax2.scatter(within_error['mtm_t'], np.abs(within_error['mtm_t'] - within_error['mtm_t_pred']), s=0.5, c='lightgreen', alpha=0.5)
    #ax2.scatter(outside_error['mtm_t'], np.abs(outside_error['mtm_t'] - outside_error['mtm_t_pred']), s=0.5, c='purple', alpha=0.5)
    ax2.scatter(within_error['mtm_t'], within_error['mtm_t'] - within_error['mtm_t_pred'], s=0.5, c='lightgreen', alpha=0.5)
    ax2.scatter(outside_error['mtm_t'], outside_error['mtm_t'] - outside_error['mtm_t_pred'], s=0.5, c='purple', alpha=0.5)
    ax2.set_xlabel('Price', fontsize=11)
    ax2.set_ylabel('Raw Errors', fontsize=11)
    ax2.legend(handles=legend_elements)
    ax2.set_title(f'{type} NN Predicted Price within Error Bars', fontsize=13)

    plt.tight_layout()
    plt.show()




In [ ]:


def plot_true_misses_and_rel_size(result, error_column='mtm_t_within_nn_error', miss_column='true_misses', rel_size_column='rel_size', type='Index'):
    """
    Plots the percentage of points falling outside the error bars for specified miss categories and rel_size distribution.

    Parameters:
    -----------
    result : DataFrame: input DataFrame containing the results data. It must have columns specified by `error_column`, `miss_column`, and `rel_size_column`.
    error_column : str, optional: name of the column indicating whether a point is within the error bars. Default is 'mtm_t_within_nn_error'.
    miss_column : str, optional: name of the column indicating the miss distances if not inside the error bar. Default is 'true_misses'.
    rel_size_column : str, optional: name of the column for rel_size data. Default is 'rel_size'.
    type : str, optional: trade type ('Index', 'Option')

    Returns:
    --------
    None
    """
    def calculate_percentages(result, error_column, miss_column):
        # Filter rows based on the specified error column
        filtered_results = result[result[error_column] == False]

        # Define boundary bands
        bands = [1, 3, 5, 10, 20, 50]
        band_labels = ['0-1%', '1-3%', '3-5%', '5-10%', '10-20%', '20-50%', '>50%']

        # Count the number of points within each band
        counts = [
            np.sum((filtered_results[miss_column] > bands[i-1]) & (filtered_results[miss_column] <= bands[i]))
            if i > 0 else np.sum(filtered_results[miss_column] <= bands[i])
            for i in range(len(bands))
        ]
        # Count points greater than the last boundary
        counts.append(np.sum(filtered_results[miss_column] > bands[-1]))

        # Calculate percentages
        total_points = len(filtered_results)
        percentages = [(count / total_points) * 100 for count in counts]

        return percentages, band_labels

    # Calculate percentages for true_misses
    percentages, band_labels = calculate_percentages(result, error_column, miss_column)

    # Define bins and labels for rel_size
    bins = [0, 1, 3, 5, 10, 20, 50, float('inf')]
    rel_size_labels = ['0-1%', '1-3%', '3-5%', '5-10%', '10-20%', '20-50%', '>50%']

    # Bin the rel_size column and calculate the percentage
    result['rel_size_bin'] = pd.cut(result[rel_size_column], bins=bins, labels=rel_size_labels, right=False)
    rel_size_percentages = result['rel_size_bin'].value_counts(normalize=True).sort_index() * 100

    # Plotting
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Plot true_misses percentages
    bars1 = axs[0].bar(range(len(percentages)), percentages, color='purple', edgecolor='black', alpha=0.7)
    for bar, percentage in zip(bars1, percentages):
        height = bar.get_height()
        axs[0].annotate(f'{percentage:.2f}%',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=9)

    axs[0].set_xticks(range(len(band_labels)))
    axs[0].set_xticklabels(band_labels)
    axs[0].set_xlabel('Outside Distance / Width Error Bar', fontsize=11)
    axs[0].set_ylabel('Percentage of Points', fontsize=11)
    axs[0].set_title(f'{type} Ground Truth Price outside Error Bars', fontsize=12)

    # Plot rel_size percentages
    bars2 = axs[1].bar(range(len(rel_size_percentages)), rel_size_percentages, color='purple', edgecolor='black', alpha=0.7)
    for bar, percentage in zip(bars2, rel_size_percentages):
        height = bar.get_height()
        axs[1].annotate(f'{percentage:.2f}%',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=9)

    axs[1].set_xticks(range(len(rel_size_labels)))
    axs[1].set_xticklabels(rel_size_labels)
    axs[1].set_xlabel('Error Bar Size / Price', fontsize=11)
    axs[1].set_ylabel('Percentage of Points', fontsize=11)
    axs[1].set_title(f'{type} Distribution of Error Bar Sizes as Percenatge of Price ', fontsize=12)

    plt.tight_layout()
    plt.show()






In [ ]:


def plot_option_dml_fit(y_true, y_pred, dydx_true, dydx_pred, name1, name2, name3, name4, name5):
    """
    Plots the predicted values against the true values and the predicted gradients against the true gradients
    for DELTA, THETA, DELTASTRIKE, VEGA 

    Parameters:
    -----------
    y_true : array: true values (ground truth)
    y_pred : array: predicted values from the model.
    dydx_true : DataFrame: true gradients (ground truth)
    dydx_pred : DataFrame: predicted gradients from the model.
    name1 : str: title for the first subplot (predicted vs true values MTM).
    name2 : str: title for the second subplot (predicted vs true gradients for DELTA).
    name3 : str: title for the third subplot (predicted vs true gradients for THETA).
    name4 : str: title for the fourth subplot (predicted vs true gradients for DELTASTRIKE).
    name5 : str: title for the fifth subplot (predicted vs true gradients for VEGA).

    Returns:
    --------
    None

    """
    plt.figure(figsize=(21, 12))

    # Plot 1: y_pred vs y_true
    plt.subplot(2, 3, 1)
    plt.scatter(y_true, y_pred, s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Values',fontsize=14)
    plt.ylabel('Predicted Values',fontsize=14)
    plt.title(f'{name1}: pred vs true',fontsize=16)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Plot 2: dydx_pred vs dydx_true (dimension 0)
    plt.subplot(2, 3, 2)
    dimension = 0
    plt.scatter(dydx_true.iloc[:, dimension], dydx_pred[:, dimension], s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Gradients',fontsize=14)
    plt.ylabel('Predicted Gradients',fontsize=14)
    plt.title(f'{name2}: pred vs true',fontsize=16)
    plt.plot([min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 
             [min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Plot 3: dydx_pred vs dydx_true (dimension 1)
    plt.subplot(2, 3, 3)
    dimension = 1
    plt.scatter(dydx_true.iloc[:, dimension], dydx_pred[:, dimension], s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Gradients',fontsize=14)
    plt.ylabel('Predicted Gradients',fontsize=14)
    plt.title(f'{name3}: pred vs true',fontsize=16)
    plt.plot([min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 
             [min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Plot 4: dydx_pred vs dydx_true (dimension 2)
    plt.subplot(2, 3, 4)
    dimension = 2
    plt.scatter(dydx_true.iloc[:, dimension], dydx_pred[:, dimension], s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Gradients',fontsize=14)
    plt.ylabel('Predicted Gradients',fontsize=14)
    plt.title(f'{name4}: pred vs true',fontsize=16)
    plt.plot([min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 
             [min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Plot 5: dydx_pred vs dydx_true (dimension 3)
    plt.subplot(2, 3, 5)
    dimension = 3
    plt.scatter(dydx_true.iloc[:, dimension], dydx_pred[:, dimension], s=1, c='cyan', label='Predicted vs True')
    plt.xlabel('True Gradients',fontsize=14)
    plt.ylabel('Predicted Gradients',fontsize=14)
    plt.title(f'{name5}: pred vs true',fontsize=16)
    plt.plot([min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 
             [min(dydx_true.iloc[:, dimension]), max(dydx_true.iloc[:, dimension])], 'r--', label='y = x')
    plt.legend()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.show()